In [1]:
import os
import pandas as pd 

In [2]:
df = pd.read_csv('t16.csv', sep = ',')
df['jjmmaa'] = pd.to_datetime(df['jjmmaa'])  

In [3]:
df.head(3)

,rgs,pr,m,js,jjmmaa,hhmm,Vitesse,To,Qualite,Q,A,Code
0,101#M1a,0,430,Ve,2016-01-01,00:00,92.0,1.0,?,421.0,101,M1a
1,102#M1b,1,9,Ve,2016-01-01,00:00,92.0,3.0,,835.0,102,M1b
2,103#M1c,1,508,Ve,2016-01-01,00:00,0.0,0.0,,0.0,103,M1c


In [4]:
vacances_hiver = pd.date_range('2016-02-06','2016-02-26')
vacances_nouvelan = pd.date_range('2016-01-01','2016-01-04')
vacances_printemps = pd.date_range('2016-04-02','2016-04-18')
vacances_ptascenssion = pd.date_range('2016-05-04','2016-05-09')
vacances_ete = pd.date_range('2016-07-05','2016-09-01')

In [5]:
df1 = df.loc[~df['jjmmaa'].isin(vacances_hiver),:]
df2 = df1.loc[~df1['jjmmaa'].isin(vacances_nouvelan),:]
df3 = df2.loc[~df2['jjmmaa'].isin(vacances_printemps),:]
df4 = df3.loc[~df3['jjmmaa'].isin(vacances_ptascenssion),:]
df5 = df4.loc[~df4['jjmmaa'].isin(vacances_ete),:]

In [6]:
df = df5

In [7]:
df.head(3)

,rgs,pr,m,js,jjmmaa,hhmm,Vitesse,To,Qualite,Q,A,Code
5496,101#M1a,0,430,Sa,2016-02-01,00:00,94.0,1.0,?,351.0,101,M1a
5497,102#M1b,1,9,Sa,2016-02-01,00:00,95.0,3.0,,845.0,102,M1b
5498,103#M1c,1,508,Sa,2016-02-01,00:00,0.0,0.0,,0.0,103,M1c


In [8]:
# Garder que les Mardis et les Jeudis
df1 = df[df['js'].str.startswith('Ma')]
df2 = df[df['js'].str.startswith('Je')]

In [9]:
df = pd.concat([df1, df2])

In [10]:
df.index = range(len(df))

In [11]:
# Pour enlever les weekends
#df = df[~df['js'].astype(str).str.startswith('Sa')]
#df = df[~df['js'].astype(str).str.startswith('Di')]

In [12]:
# L'heure de pointe du matin de 7.30 a 9h
cond1 = df.apply ( lambda row : row["hhmm"] > '07:54', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '09:06', axis = 1 )
# L'intersection des deux condition.
df1 = pd.merge(df[cond1], df[cond2], how='inner')

In [13]:
# de 9h a 10h
cond1 = df.apply ( lambda row : row["hhmm"] > '08:54', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '10:06', axis = 1 )
# L'intersection des deux condition.
df2 = pd.merge(df[cond1], df[cond2], how='inner')

In [14]:
# L'heure de pointe du matin
cond1 = df.apply ( lambda row : row["hhmm"] > '11:54', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '13:06', axis = 1 )
# L'intersection des deux condition.
df3 = pd.merge(df[cond1], df[cond2], how='inner')

In [15]:
# L'heure de pointe du Soir de 17 a 19
cond1 = df.apply ( lambda row : row["hhmm"] > '17:00', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '19:06', axis = 1 )
# L'intersection des deux condition.
df4 = pd.merge(df[cond1], df[cond2], how='inner')

## La moyenne de la vitesse et le debit des heures de pointes

In [45]:
vitesse_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').mean()
vitesse_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').mean()
vitesse_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').mean()
vitesse_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').mean()

In [46]:
VQ = pd.concat([vitesse_8_9,vitesse_9_10,vitesse_12_13,vitesse_17_19], axis = 1)
VQ.columns = ['V_8_9','Q_8_9','V_9_10','Q_9_10','V_12_13','Q_12_13','V_17_19','Q_17_19']

In [47]:
# Reindexer la table
VQ['Code'] = VQ.index
VQ.index = range(len(VQ))

In [48]:
# table isidor
df_isidor = pd.read_csv("table3.csv",header=None, sep = ';')
# renommer la colonne 
df_isidor.columns = ['Code']
# renommer qlq stations
df_isidor['Code'] = df_isidor['Code'].str.replace('MM713.','M7')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM613.','M6')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM513.','M5')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM413.','M4')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM113.','M1')

In [49]:
#L'union Left
result = pd.merge(df_isidor, VQ, how = 'left')

In [50]:
len(result) == len(df_isidor)

True

## Les déciles des debits des heures de pointe

In [16]:
decile1_8_9 = df1[['Code','Q']].groupby('Code').quantile(0.8)
decile1_9_10 = df2[['Code','Q']].groupby('Code').quantile(0.8)
decile1_12_13 = df3[['Code','Q']].groupby('Code').quantile(0.8)
decile1_17_19 = df4[['Code','Q']].groupby('Code').quantile(0.8)

decile2_8_9 = df1[['Code','Q']].groupby('Code').quantile(0.9)
decile2_9_10 = df2[['Code','Q']].groupby('Code').quantile(0.9)
decile2_12_13 = df3[['Code','Q']].groupby('Code').quantile(0.9)
decile2_17_19 = df4[['Code','Q']].groupby('Code').quantile(0.9)

In [18]:
Q_deciles89 = pd.concat([decile1_8_9,decile1_9_10,decile1_12_13,decile1_17_19,decile2_8_9,decile2_9_10
                        ,decile2_12_13,decile2_17_19], axis = 1)
Q_deciles89.columns = ['Q1_8_9','Q1_9_10','Q1_12_13','Q1_17_19'
                      ,'Q2_8_9','Q2_9_10','Q2_12_13','Q2_17_19']

In [19]:
# Reindexer la table
Q_deciles89['Code'] = Q_deciles89.index
Q_deciles89.index = range(len(Q_deciles89))

In [24]:
#L'union Left
result_deciles89 = pd.merge(df_isidor, Q_deciles89, how = 'left')

In [25]:
# Exporter la table
result_deciles89.to_csv('Q_deciles89.csv', index = False, encoding = 'utf-8')

## Ajouter la colonne sens

In [51]:
# Creaons d'abords la table S qui contient le sens 1 ou 2. 

# Creer une liste vide
d=[]
tab=result.values
for i in range(len(result)):
    s = tab[i,0][2]
    if (s.islower()):
        d.append('Sens 2')
    elif (s.isupper()):
        d.append('Sens 1')
    else:
        d.append(0)

In [52]:
# Colonne Contient les modalites (Sens1, Sens2) qu'on doit ajouter.
f = pd.DataFrame({'Sens':d})

In [53]:
f.head()

,Sens
0,Sens 1
1,Sens 1
2,Sens 1
3,Sens 1
4,Sens 1


In [54]:
result.head()

,Code,V_8_9,Q_8_9,V_9_10,Q_9_10,V_12_13,Q_12_13,V_17_19,Q_17_19
0,M7I,78.905405,4384.716216,81.280000,3957.866667,87.547945,3588.287671,83.207792,4603.844156
1,M7J,80.864865,4191.297297,82.080000,3658.986667,89.219178,3168.205479,85.038961,4471.610390
2,M7N,40.810811,764.067568,42.386667,739.600000,42.684932,700.328767,34.584416,866.649351
3,M7O,94.067568,4291.256757,95.306667,3965.040000,100.890411,3657.191781,86.779221,4694.285714
4,M7P,37.959459,1581.648649,38.346667,1400.480000,34.958904,1058.424658,30.636364,1674.246753


In [55]:
# test
len(f) == len(result)

True

In [56]:
rslt = pd.concat([result, f], axis=1)
rslt.head()

,Code,V_8_9,Q_8_9,V_9_10,Q_9_10,V_12_13,Q_12_13,V_17_19,Q_17_19,Sens
0,M7I,78.905405,4384.716216,81.280000,3957.866667,87.547945,3588.287671,83.207792,4603.844156,Sens 1
1,M7J,80.864865,4191.297297,82.080000,3658.986667,89.219178,3168.205479,85.038961,4471.610390,Sens 1
2,M7N,40.810811,764.067568,42.386667,739.600000,42.684932,700.328767,34.584416,866.649351,Sens 1
3,M7O,94.067568,4291.256757,95.306667,3965.040000,100.890411,3657.191781,86.779221,4694.285714,Sens 1
4,M7P,37.959459,1581.648649,38.346667,1400.480000,34.958904,1058.424658,30.636364,1674.246753,Sens 1


In [57]:
# Exporter la table Debit vitesse mardi jeudi
rslt.to_csv('VQ_MJ.csv', index = False, encoding = 'utf-8')

# Les déciles des debits des heures de pointe par sens


In [26]:
decile1_8_9 = df1[['Code','Q']].groupby('Code').quantile(0.8)
decile1_9_10 = df2[['Code','Q']].groupby('Code').quantile(0.8)
decile1_12_13 = df3[['Code','Q']].groupby('Code').quantile(0.8)
decile1_17_19 = df4[['Code','Q']].groupby('Code').quantile(0.8)

decile2_8_9 = df1[['Code','Q']].groupby('Code').quantile(0.9)
decile2_9_10 = df2[['Code','Q']].groupby('Code').quantile(0.9)
decile2_12_13 = df3[['Code','Q']].groupby('Code').quantile(0.9)
decile2_17_19 = df4[['Code','Q']].groupby('Code').quantile(0.9)

In [27]:
Q_deciles89 = pd.concat([decile1_8_9,decile1_9_10,decile1_12_13,decile1_17_19,decile2_8_9,decile2_9_10
                        ,decile2_12_13,decile2_17_19], axis = 1)
Q_deciles89.columns = ['Q1_8_9','Q1_9_10','Q1_12_13','Q1_17_19'
                      ,'Q2_8_9','Q2_9_10','Q2_12_13','Q2_17_19']

In [28]:
# Reindexer la table
Q_deciles89['Code'] = Q_deciles89.index
Q_deciles89.index = range(len(Q_deciles89))

In [29]:
#L'union Left
result_deciles89 = pd.merge(df_isidor, Q_deciles89, how = 'left')

In [32]:
# Creaons d'abords la table S qui contient le sens 1 ou 2. 

# Creer une liste vide
d=[]
tab=result_deciles89.values
for i in range(len(result_deciles89)):
    s = tab[i,0][2]
    if (s.islower()):
        d.append('Sens 2')
    elif (s.isupper()):
        d.append('Sens 1')
    else:
        d.append(0)

In [33]:
# Colonne Contient les modalites (Sens1, Sens2) qu'on doit ajouter.
f = pd.DataFrame({'Sens':d})

In [34]:
rslt2 = pd.concat([result_deciles89, f], axis=1)
rslt2.head()

,Code,Q1_8_9,Q1_9_10,Q1_12_13,Q1_17_19,Q2_8_9,Q2_9_10,Q2_12_13,Q2_17_19,Sens
0,M7I,5058.4,4804.6,3692.8,5230.8,5201.0,4915.4,3803.0,5353.0,Sens 1
1,M7J,5263.4,4975.8,3814.4,5504.2,5461.8,5135.8,3911.0,5583.0,Sens 1
2,M7N,1959.2,1806.6,1681.8,1997.8,2123.7,2038.0,1867.8,2367.6,Sens 1
3,M7O,4683.2,4527.6,3684.8,5348.4,4755.5,4634.2,3738.4,5491.6,Sens 1
4,M7P,4459.4,3725.6,3398.6,4898.4,4607.8,4402.6,3623.2,5217.8,Sens 1


In [72]:
# Exporter la table
rslt2.to_csv('Q_deciles.csv', index = False, encoding = 'utf-8')

# Les déciles des vitesses des heures de pointe par sens


In [35]:
decile1_8_9 = df1[['Code','Vitesse']].groupby('Code').quantile(0.1)
decile1_9_10 = df2[['Code','Vitesse']].groupby('Code').quantile(0.1)
decile1_12_13 = df3[['Code','Vitesse']].groupby('Code').quantile(0.1)
decile1_17_19 = df4[['Code','Vitesse']].groupby('Code').quantile(0.1)

decile2_8_9 = df1[['Code','Vitesse']].groupby('Code').quantile(0.2)
decile2_9_10 = df2[['Code','Vitesse']].groupby('Code').quantile(0.2)
decile2_12_13 = df3[['Code','Vitesse']].groupby('Code').quantile(0.2)
decile2_17_19 = df4[['Code','Vitesse']].groupby('Code').quantile(0.2)

In [38]:
V_deciles12 = pd.concat([decile1_8_9,decile1_9_10,decile1_12_13,decile1_17_19,decile2_8_9,decile2_9_10
                        ,decile2_12_13,decile2_17_19], axis = 1)
V_deciles12.columns = ['V1_8_9','V1_9_10','V1_12_13','V1_17_19'
                      ,'V2_8_9','V2_9_10','V2_12_13','V2_17_19']

In [39]:
# Reindexer la table
V_deciles12['Code'] = V_deciles12.index
V_deciles12.index = range(len(V_deciles12))

In [40]:
#L'union Left
result_deciles12 = pd.merge(df_isidor, V_deciles12, how = 'left')

In [41]:
# Creaons d'abords la table S qui contient le sens 1 ou 2. 

# Creer une liste vide
d=[]
tab=result_deciles12.values
for i in range(len(result_deciles12)):
    s = tab[i,0][2]
    if (s.islower()):
        d.append('Sens 2')
    elif (s.isupper()):
        d.append('Sens 1')
    else:
        d.append(0)

In [42]:
# Colonne Contient les modalites (Sens1, Sens2) qu'on doit ajouter.
f = pd.DataFrame({'Sens':d})

In [43]:
rslt3 = pd.concat([result_deciles12, f], axis=1)
rslt3.head()

,Code,V1_8_9,V1_9_10,V1_12_13,V1_17_19,V2_8_9,V2_9_10,V2_12_13,V2_17_19,Sens
0,M7I,67.8,68.4,87.0,77.6,75.6,80.0,88.0,82.0,Sens 1
1,M7J,63.3,60.6,90.2,80.2,80.0,85.8,93.0,87.0,Sens 1
2,M7N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sens 1
3,M7O,85.3,82.8,102.0,51.8,96.0,98.0,103.0,73.6,Sens 1
4,M7P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sens 1


In [44]:
# Exporter la table
rslt3.to_csv('V_deciles.csv', index = False, encoding = 'utf-8')